# IndustriALL AI Challenge

---


## Reading Data


In [ ]:
from multiprocessing import cpu_count
import os
import time
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
DATAFOLDER = '../data/'
BASE_NAME = 'TAG_iALL_PS_'
full_csv_exists = 'full.csv' in os.listdir(DATAFOLDER)

In [ ]:
if full_csv_exists:
    start_time = time.time()
    full_df = pd.read_csv('../data/full.csv')
    end_time = time.time()
    print('Elapsed time: ' + str(end_time - start_time) + ' seconds')

else:
    start_time = time.time()
    def read(file):
        if file.endswith('00.csv'):
            return None

        print('Reading file: ' + file)
        df = pd.read_csv(DATAFOLDER + file)
        df = df.set_index('timestamp')

        return df[file.split('.')[0]]

    files_to_process = [file for file in os.listdir(DATAFOLDER) if not file.endswith('00.csv')]

    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor(cpu_count()) as executor:
        # Map the function to process each file in parallel
        dfs = list(executor.map(read, files_to_process))

    # Create the full_df by concatenating the DataFrames
    full_df = pd.concat([pd.read_csv(DATAFOLDER + 'TAG_iALL_PS_00.csv').set_index('timestamp')] + dfs, ignore_index=False, axis=1)

    # Rename the 'target_iALL_PS' column to 'status' 
    if 'target_iALL_PS.csv' in files_to_process:
        full_df = full_df.rename(columns={'target_iALL_PS': 'status'})

    # Save the result to a CSV file
    full_df.to_csv(DATAFOLDER + 'full.csv')
    end_time = time.time()
    print('Elapsed time: ' + str(end_time - start_time) + ' seconds')
    

---

## Filtering and Cleaning Data

In [ ]:
count = 0
for col in full_df.columns:
    print(f"{col} : {full_df[col].dtype}", end=" | ")
    count += 1
    if count % 5 == 0:
        print()

In [ ]:
# Drop duplicates and NaNs (when all rows/columns are NaN)
full_df = full_df.drop_duplicates()
full_df = full_df.dropna(axis=1, how='all')
full_df = full_df.dropna(axis=0, how='all')
full_df = full_df.reset_index()

# Convert the timestamp column to datetime and set it as the index
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])
full_df = full_df.sort_values(by='timestamp')
full_df = full_df.reset_index()
full_df = full_df.set_index('timestamp') if 'timestamp' in full_df.columns else full_df

# Create a column with the status as a boolean (might be useful for plotting later)
full_df['status_bool'] = np.where(full_df['status'] == 'NORMAL', 0, 1)

# drop the columns that are not useful
full_df = full_df.drop(columns=['index'])
full_df = full_df.drop(columns=['level_0'])

---

## Exploratory Analysis

### Functions to find and return outliers

In [ ]:
def find_outliers_in_series(series:pd.Series):
    """Find outliers in a pandas Series using the interquartile range (IQR) method.

    Args:
        series (pd.Series): The Series to find outliers in.

    Returns:
        _type_: A Series with the outliers.
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    # Calculate the upper and lower bounds
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    df_cpy = series.copy()
    
    # set not outliers to NaN
    df_cpy[(df_cpy >= lower_bound) & (df_cpy <= upper_bound)] = np.nan

    return df_cpy

def find_outliers(df:pd.Series | pd.DataFrame) -> pd.DataFrame:
    """Find outliers in a pandas Series or DataFrame using the interquartile range (IQR) method.

    Args:
        df (pd.Series | pd.DataFrame): The Series or DataFrame to find outliers in.

    Returns:
        pd.DataFrame: A DataFrame with the outliers.
    """
    all_columns = df.columns if isinstance(df, pd.DataFrame) else [df.name]
    outliers = pd.DataFrame()
    for column in all_columns:
        if df[column].dtype == 'object' or column == 'status':
            continue
        outliers[column] = find_outliers_in_series(df[column])
            
    return outliers

In [ ]:
print(full_df['status'].value_counts())
print(full_df['status'].value_counts(normalize=True))

# drop rows with all NaN values
outliers = find_outliers(full_df)
outliers = outliers.dropna(how='all') 
print(f"Outliers: {outliers.shape[0]}")

# build dataframes with only ANORMAL status
non_normal_df = full_df[full_df['status'] == 'ANORMAL']
print(f"Non normal: {non_normal_df.shape[0]}")

# drop rows with all NaN values
non_normal_and_outlier = find_outliers(non_normal_df)
non_normal_and_outlier = non_normal_and_outlier.dropna(how='all') 
print(f"Non normal and outlier: {non_normal_and_outlier.shape[0]}")


In [ ]:
# Pratical exemple of the code above using TAG 00 as example

print("Full TAG 00 data:")
print(full_df['TAG_iALL_PS_00'].describe())

print("\nNon normal TAG 00 data:")
print(non_normal_df['TAG_iALL_PS_00'].describe())

print("\nOutliers in TAG 00:")
print(outliers['TAG_iALL_PS_00'].describe())

print("\nOutliers in non normal TAG 00:")
print(non_normal_and_outlier['TAG_iALL_PS_00'].describe())


In [ ]:

# boxplot of TAG_iALL_PS_00 (example)
fig = px.box(full_df, y='TAG_iALL_PS_00', color='status')
fig.update_layout(title='TAG_iALL_PS_00 boxplot', yaxis_title='TAG_iALL_PS_00')
fig.show()

---

### Check the correlation between features

In [ ]:
# full_df.corr(numeric_only=True)
fig, ax = plt.subplots(figsize=(60,30))
sns.heatmap(full_df.corr(numeric_only=True), annot=True, ax=ax)

In [ ]:
corr_status_1 = full_df.corr('pearson', numeric_only=True)['status_bool'].sort_values(ascending=False)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr_status_1.to_frame(), annot=True, ax=ax)

## Machine Learning Models

### Naive Bayes using Gaussian/Bernoulli Distribution

#### Bayes Theorem



> P(A|B) = (P(B|A) * P(A)) / P(B)

1) So, in our case:

    - P(y|X) = (P(X|y) * P(y)) / P(X)
    - y is the class labels tath we want to predict and X is the feature vector given by (X=x1, x2, x3, ...m xn)

    Then, we assume that features are mutually independent
    So, addume that, we can split it into different componenets:
    - P(y|X) = (P(x1|y) * P(x2|y) * ... * P(xn|y) * P(y)) / P(X)

<br>

2) Select the class with highest posterior probability

    - y = argmax(y)P(y|X) = argmax(y)(P(x1|y) * P(x2|y) * ... * P(xn|y) * P(y)) / P(X)
    - to simplify the formula, we can throw away the P(X) because this not depends os y at all:
        - y = argmax(y)P(y|X) = argmax(y)(P(x1|y) * P(x2|y) * ... * P(xn|y) * P(y))

    - all the probabilities are values between 0 and 1, so we can tranform it in a sum o log(k) values:
        - y = argmax(y)P(y|X) = argmax(y)(log(P(x1|y)) + log(P(x2|y)) + ... + log(P(xn|y)) + log(P(y)))

<br>

3) Prior and class conditional

    - P(y) is the Prior probability ---> frequency of each class
    - P(xi|y) is the class conditional probability ---> Model with Gaussian

    - ![image.png](attachment:image.png)

4) Main Steps
    - Training:
        - Calculate mean, var and prior (frequency) of each class
    - Predictions:
        - Calculate posterior for each class with: 
        > y = argmax(y)P(y|X) = argmax(y)(log(P(x1|y)) + log(P(x2|y)) + ... + log(P(xn|y)) + log(P(y)))
    - Choose the highest posterior probability

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# set nan values to 0. The missing values might not affect the model
full_df = full_df.fillna(0)

# create a list of features
features = full_df.columns.tolist()
features.remove('status')
features.remove('status_bool')

# create a list of target
target = 'status'

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(full_df[features], full_df[target], test_size=0.2, random_state=42)

# create a Gaussian and Bernoulli Naive Bayes classifier
gnb = GaussianNB()
bnb = BernoulliNB()

# train the model using the training sets
gnb.fit(X_train, y_train)
bnb.fit(X_train, y_train)

# predict the response for test dataset
y_pred_gnb = gnb.predict(X_test)
y_pred_bnb = bnb.predict(X_test)

# print the first 10 anormal predictions
def print_result_sample(y_pred, y_test):
    first_10_anormal_predictions = []
    first_10_anormal_actual = []
    for i in range(len(y_pred)):
        if y_pred[i] == 'ANORMAL':
            first_10_anormal_predictions.append(y_pred[i])
            first_10_anormal_actual.append(y_test[i])
            if len(first_10_anormal_predictions) == 10:
                break

    print(first_10_anormal_predictions)
    print(first_10_anormal_actual)

print(f"\nGNB accuracy: {accuracy_score(y_test, y_pred_gnb)}")
print_result_sample(y_pred_gnb, y_test)
print(f"\nBNB accuracy: {accuracy_score(y_test, y_pred_bnb)}")
print_result_sample(y_pred_bnb, y_test)

# check how many predictions match between the two models
print(f"\nNumber of matching predictions: {np.sum(y_pred_gnb == y_pred_bnb)} / {len(y_pred_gnb)}")
